In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

In [2]:
df = pd.read_csv('D:/Data Sience/Project/Dt_02/clean data/Clean_data.csv')

In [3]:
df.dropna()

,DIABETE3,SEX,_AGEG5YR,EDUCA,_BMI5,_BMI5CAT,GENHLTH,PHYSHLTH,_TOTINDA,EXERANY2,...,SMOKDAY2,_RFSMOK3,DRNKANY5,ALCDAY5,AVEDRNK2,DRNK3GE5,_RFBING5,CVDSTRK3,CVDINFR4,CVDCRHD4
0,0,2,9,4,1855,2,4,88,2,2,...,1,2,1,103,2,88,1,2,2,2
1,0,2,14,6,1695,1,2,88,1,1,...,3,1,1,101,1,88,1,2,2,2
2,0,2,13,6,2141,2,2,88,2,2,...,3,1,1,107,2,88,1,2,2,2
3,0,1,8,6,2585,3,2,2,1,1,...,3,1,1,210,2,88,1,2,2,2
4,0,1,10,4,2510,3,2,88,1,1,...,3,1,1,215,4,88,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512818,0,1,3,3,2647,3,2,88,1,1,...,3,1,1,201,24,1,2,2,2,2
512819,0,2,11,4,2780,3,3,88,2,2,...,3,1,1,204,2,88,1,2,2,2
512820,0,1,1,5,3797,4,1,88,1,1,...,1,2,1,203,8,3,2,2,2,2
512821,0,1,9,2,2403,2,2,88,2,2,...,1,2,1,103,18,3,2,2,2,2


In [4]:
x = df.drop('DIABETE3', axis= 1)
y = df['DIABETE3']

In [5]:
y.value_counts(normalize=True)

DIABETE3
0    0.904326
1    0.095674
Name: proportion, dtype: float64

In [6]:
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)

In [8]:
imputer = SimpleImputer(strategy='mean')

In [9]:
x_train_imputed = imputer.fit_transform(x_train)
x_test_imputed = imputer.transform(x_test)

In [10]:
smote = SMOTE(random_state=42)
x_train_res, y_train_res = smote.fit_resample(x_train_imputed, y_train)

In [11]:
rf_classifier = RandomForestClassifier(bootstrap=True, class_weight="balanced_subsample", random_state=42)

In [12]:
rf_classifier.fit(x_train_res, y_train_res)


RandomForestClassifier(class_weight='balanced_subsample', random_state=42)

In [23]:
from sklearn.preprocessing import StandardScaler, Binarizer

In [24]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [34]:
param_grid = {
    'n_estimators': np.arange(100, 1001, 100),
    'max_depth': [None] + list(np.arange(10, 101, 10)),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [37]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [26]:
rf = RandomForestClassifier(class_weight='balanced_subsample')

In [31]:
from sklearn.model_selection import RandomizedSearchCV

In [38]:
rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    n_iter=10,  
    cv=skf,
    verbose=2,
    random_state=42,
    n_jobs=1
)

In [39]:
rf_random

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=RandomForestClassifier(class_weight='balanced_subsample'),
                   n_jobs=1,
                   param_distributions={'max_depth': [None, 10, 20, 30, 40, 50,
                                                      60, 70, 80, 90, 100],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': array([ 100,  200,  300,  400,  500,  600,  700,  800,  900, 1000])},
                   random_state=42, verbose=2)

In [40]:
rf_random.fit(x_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END max_depth=40, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=500; total time=16.3min
[CV] END max_depth=40, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=500; total time=18.1min
[CV] END max_depth=40, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimator

c:\Users\mnema\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
30 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\mnema\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\mnema\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\mnema\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=RandomForestClassifier(class_weight='balanced_subsample'),
                   n_jobs=1,
                   param_distributions={'max_depth': [None, 10, 20, 30, 40, 50,
                                                      60, 70, 80, 90, 100],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': array([ 100,  200,  300,  400,  500,  600,  700,  800,  900, 1000])},
                   random_state=42, verbose=2)

In [42]:
best_rf = rf_random.best_estimator_

In [43]:
y_pred_proba = best_rf.predict_proba(x_test)[:, 1]

In [44]:
binarizer = Binarizer(threshold=0.5)

In [45]:
y_pred_binary = binarizer.fit_transform(y_pred_proba.reshape(-1, 1)).ravel()

In [46]:
y_test_binary = binarizer.fit_transform(y_test.values.reshape(-1, 1)).ravel()

In [47]:
conf_matrix = confusion_matrix(y_test_binary, y_pred_binary)
class_report = classification_report(y_test_binary, y_pred_binary)

In [49]:
print("Best Parameters from RandomizedSearchCV:\n", rf_random.best_params_)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

Best Parameters from RandomizedSearchCV:
 {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 40}

Confusion Matrix:
 [[87140  5612]
 [ 6804  3009]]

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.94      0.93     92752
           1       0.35      0.31      0.33      9813

    accuracy                           0.88    102565
   macro avg       0.64      0.62      0.63    102565
weighted avg       0.87      0.88      0.88    102565



In [54]:
print(f"Best Hyperparameters: {rf_random.best_params_}")

Best Hyperparameters: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 40}


In [56]:
min_proba = y_pred_proba.min()
max_proba = y_pred_proba.max()

In [57]:
thresholds = np.arange(min_proba, max_proba, 0.01)

In [58]:
precision_scores = []
recall_scores = []

In [60]:
from sklearn.metrics import  precision_score, recall_score, f1_score


In [61]:
for t in thresholds:

    y_pred_threshold = np.where(y_pred_proba > t, 1, 0)


    precision = precision_score(y_test, y_pred_threshold)
    recall = recall_score(y_test, y_pred_threshold)


    precision_scores.append((t, precision))
    recall_scores.append((t, recall))

    print(f"Threshold: {t:.2f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

Threshold: 0.00, Precision: 0.0957, Recall: 1.0000
Threshold: 0.01, Precision: 0.1023, Recall: 0.9972
Threshold: 0.02, Precision: 0.1094, Recall: 0.9907
Threshold: 0.03, Precision: 0.1162, Recall: 0.9837
Threshold: 0.04, Precision: 0.1225, Recall: 0.9781
Threshold: 0.05, Precision: 0.1285, Recall: 0.9711
Threshold: 0.06, Precision: 0.1342, Recall: 0.9631
Threshold: 0.07, Precision: 0.1395, Recall: 0.9544
Threshold: 0.08, Precision: 0.1450, Recall: 0.9457
Threshold: 0.09, Precision: 0.1501, Recall: 0.9362
Threshold: 0.10, Precision: 0.1547, Recall: 0.9250
Threshold: 0.11, Precision: 0.1594, Recall: 0.9138
Threshold: 0.12, Precision: 0.1642, Recall: 0.9012
Threshold: 0.13, Precision: 0.1692, Recall: 0.8895
Threshold: 0.14, Precision: 0.1739, Recall: 0.8757
Threshold: 0.15, Precision: 0.1789, Recall: 0.8630
Threshold: 0.16, Precision: 0.1836, Recall: 0.8486
Threshold: 0.17, Precision: 0.1884, Recall: 0.8347
Threshold: 0.18, Precision: 0.1934, Recall: 0.8199
Threshold: 0.19, Precision: 0.1